In [1]:
!pip install google-api-python-client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 4.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import math

# Define a function to calculate total quota usage for a given number of videos
def calculate_quota_usage(num_videos):
    # Quota for video search
    video_search_requests = math.ceil(num_videos / 50)
    video_search_quota = video_search_requests * 500
    
    # Quota for comments (1 unit per video to get top 100 comments)
    comments_quota = num_videos * 1
    
    # Total quota usage
    total_quota = video_search_quota + comments_quota
    
    return total_quota

# Now, find the maximum number of videos that keep total quota under 9000 units
target_quota = 9000
max_videos = 0

for videos in range(1, 5000):  # Try different values for videos
    total_quota = calculate_quota_usage(videos)
    if total_quota <= target_quota:
        max_videos = videos
    else:
        break

max_videos, calculate_quota_usage(max_videos)  # Return the number of videos and the total quota usage

(800, 8800)

In [6]:
import pandas as pd
import os
import time
import logging
from googleapiclient.discovery import build
import config  # Import the config file for API key
from googleapiclient.errors import HttpError

# Set up your YouTube API key
api_key = config.YOUTUBE_API_KEY

# Build the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Set up logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Ensure a directory exists to store the files
output_folder = 'youtube_data'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Function to get video details by a search term (with pagination, likes, and error handling)
def get_video_details(search_query, max_results=800):
    videos = []
    next_page_token = None

    logging.info(f"Starting to scrape video details for search term: {search_query}")

    while len(videos) < max_results:
        try:
            # Step 1: Search for videos using the search term
            request = youtube.search().list(
                q=f'{search_query} 2024 "presidential election" OR "US election" OR "US elections"',
                part='snippet',
                type='video',
                maxResults=min(50, max_results - len(videos)),  # Fetch up to 50 results per request
                pageToken=next_page_token,
                relevanceLanguage="en",  # Filter by English content
                regionCode="US",  # Prioritize US-based content
                order="relevance",  # Order results by relevance to the search term
                safeSearch="moderate",  # Optional: filter out inappropriate content
            )
            response = request.execute()

            video_ids = [item['id']['videoId'] for item in response['items']]  # Collect video IDs

            # Step 2: Fetch statistics (likes, views, etc.) for the collected video IDs
            stats_request = youtube.videos().list(
                part='statistics',
                id=','.join(video_ids)  # Pass video IDs as a comma-separated string
            )
            stats_response = stats_request.execute()

            # Step 3: Combine snippet and statistics data
            for item, stats in zip(response['items'], stats_response['items']):
                video = {
                    'video_id': item['id']['videoId'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_title': item['snippet']['channelTitle'],
                    'like_count': stats['statistics'].get('likeCount', 0),  # Likes
                    'view_count': stats['statistics'].get('viewCount', 0)   # Views (optional)
                }

                # Filter to include only videos explicitly mentioning the election
                if 'election' in video['title'].lower() or 'election' in video['description'].lower():
                    videos.append(video)

            logging.info(f"Scraped {len(videos)} videos so far...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except Exception as e:
            logging.error(f"Error occurred while fetching videos: {e}")
            time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping video details. Total videos scraped: {len(videos)}")
    return pd.DataFrame(videos)

# Function to get top 500 comments and their replies from a video, including parent-child relationships
def get_video_comments(video_id, max_results=500):
    comments = []
    next_page_token = None

    logging.info(f"Starting to scrape comments for video: {video_id}")

    while len(comments) < max_results:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",  # Include replies in the response
                videoId=video_id,
                maxResults=min(100, max_results - len(comments)),  # Fetch up to 100 comments per request
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                # Extract top-level comment
                top_level_comment = {
                    'comment_id': item['snippet']['topLevelComment']['id'],
                    'author': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                    'text': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                    'published_at': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                    'like_count': item['snippet']['topLevelComment']['snippet']['likeCount'],
                    'is_reply': False,  # This indicates it's a top-level comment
                    'parent_id': video_id  # For top-level comments, the parent is the video
                }
                comments.append(top_level_comment)

                # If there are replies, extract them and track parent comment
                if 'replies' in item:
                    for reply in item['replies']['comments']:
                        reply_comment = {
                            'comment_id': reply['id'],
                            'author': reply['snippet']['authorDisplayName'],
                            'text': reply['snippet']['textDisplay'],
                            'published_at': reply['snippet']['publishedAt'],
                            'like_count': reply['snippet']['likeCount'],
                            'is_reply': True,  # This indicates it's a reply to a top-level comment
                            'parent_id': top_level_comment['comment_id']  # This is the parent comment ID
                        }
                        comments.append(reply_comment)

            logging.info(f"Scraped {len(comments)} comments so far for video {video_id}...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except HttpError as e:
            # Check if the error is due to comments being disabled
            if e.resp.status == 403 and 'commentsDisabled' in str(e):
                logging.warning(f"Encountered 403 Forbidden with reason 'commentsDisabled' for video {video_id}")
                break  # Do not retry for this video
            else:
                logging.error(f"Error occurred while fetching comments for video {video_id}: {e}")
                time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping comments for video {video_id}. Total comments: {len(comments)}")
    return pd.DataFrame(comments)

# Example: Scrape video details for a search term (up to 800 videos)
search_term = 'U.S. elections 2024'
try:
    videos_df = get_video_details(search_term, max_results=800)
    video_file_path = os.path.join(output_folder, 'youtube_videos.csv')
    videos_df.to_csv(video_file_path, index=False)
    logging.info(f"Video details saved to: {video_file_path}")
except Exception as e:
    logging.error(f"Error occurred while saving video details: {e}")

# Scrape top 500 comments (including replies) for **each video** in the list of scraped videos
for idx, row in videos_df.iterrows():
    try:
        video_id = row['video_id']
        comments_df = get_video_comments(video_id, max_results=500)
        comments_file_path = os.path.join(output_folder, f'youtube_comments_{video_id}.csv')
        comments_df.to_csv(comments_file_path, index=False)
        logging.info(f"Comments for video {video_id} saved to: {comments_file_path}")
    except Exception as e:
        logging.error(f"Error occurred while saving comments for video {video_id}: {e}")

2024-10-16 15:11:08,654 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-10-16 15:11:08,669 - INFO - Starting to scrape video details for search term: U.S. elections 2024
2024-10-16 15:11:09,796 - INFO - Scraped 44 videos so far...
2024-10-16 15:11:10,735 - INFO - Scraped 88 videos so far...
2024-10-16 15:11:11,461 - INFO - Scraped 121 videos so far...
2024-10-16 15:11:12,202 - INFO - Scraped 160 videos so far...
2024-10-16 15:11:12,996 - INFO - Scraped 199 videos so far...
2024-10-16 15:11:13,803 - INFO - Scraped 239 videos so far...
2024-10-16 15:11:14,618 - INFO - Scraped 284 videos so far...
2024-10-16 15:11:15,647 - INFO - Scraped 328 videos so far...
2024-10-16 15:11:16,611 - INFO - Scraped 374 videos so far...
2024-10-16 15:11:17,381 - INFO - Scraped 421 videos so far...
2024-10-16 15:11:18,405 - INFO - Scraped 467 videos so far...
2024-10-16 15:11:19,534 - INFO - Scraped 495 videos so far...
2024-10-16 15:11:19,535 - INFO - Finished scraping video details. Tot

2024-10-16 15:11:28,405 - INFO - Starting to scrape comments for video: M2vtdBh0YHc
2024-10-16 15:11:28,880 - INFO - Scraped 107 comments so far for video M2vtdBh0YHc...
2024-10-16 15:11:29,272 - INFO - Scraped 260 comments so far for video M2vtdBh0YHc...
2024-10-16 15:11:29,273 - INFO - Finished scraping comments for video M2vtdBh0YHc. Total comments: 260
2024-10-16 15:11:29,280 - INFO - Comments for video M2vtdBh0YHc saved to: youtube_data/youtube_comments_M2vtdBh0YHc.csv
2024-10-16 15:11:29,281 - INFO - Starting to scrape comments for video: s_NeINDX1C8
2024-10-16 15:11:29,675 - INFO - Scraped 112 comments so far for video s_NeINDX1C8...
2024-10-16 15:11:30,441 - INFO - Scraped 264 comments so far for video s_NeINDX1C8...
2024-10-16 15:11:30,859 - INFO - Scraped 418 comments so far for video s_NeINDX1C8...
2024-10-16 15:11:31,228 - INFO - Scraped 598 comments so far for video s_NeINDX1C8...
2024-10-16 15:11:31,229 - INFO - Finished scraping comments for video s_NeINDX1C8. Total comm

2024-10-16 15:11:39,789 - INFO - Comments for video NPOjxvFYstQ saved to: youtube_data/youtube_comments_NPOjxvFYstQ.csv
2024-10-16 15:11:39,790 - INFO - Starting to scrape comments for video: 5p2AmViVwKI
2024-10-16 15:11:40,086 - INFO - Scraped 126 comments so far for video 5p2AmViVwKI...
2024-10-16 15:11:40,617 - INFO - Scraped 278 comments so far for video 5p2AmViVwKI...
2024-10-16 15:11:41,023 - INFO - Scraped 452 comments so far for video 5p2AmViVwKI...
2024-10-16 15:11:41,389 - INFO - Scraped 564 comments so far for video 5p2AmViVwKI...
2024-10-16 15:11:41,391 - INFO - Finished scraping comments for video 5p2AmViVwKI. Total comments: 564
2024-10-16 15:11:41,401 - INFO - Comments for video 5p2AmViVwKI saved to: youtube_data/youtube_comments_5p2AmViVwKI.csv
2024-10-16 15:11:41,402 - INFO - Starting to scrape comments for video: n4aRCA1qxnk
2024-10-16 15:11:41,835 - INFO - Scraped 105 comments so far for video n4aRCA1qxnk...
2024-10-16 15:11:42,220 - INFO - Scraped 222 comments so fa

2024-10-16 15:11:52,412 - INFO - Starting to scrape comments for video: 4CkK2JwZVjw
2024-10-16 15:11:52,663 - INFO - Scraped 22 comments so far for video 4CkK2JwZVjw...
2024-10-16 15:11:52,664 - INFO - Finished scraping comments for video 4CkK2JwZVjw. Total comments: 22
2024-10-16 15:11:52,669 - INFO - Comments for video 4CkK2JwZVjw saved to: youtube_data/youtube_comments_4CkK2JwZVjw.csv
2024-10-16 15:11:52,670 - INFO - Starting to scrape comments for video: rWpuh6dTiyY
2024-10-16 15:11:52,844 - INFO - Scraped 6 comments so far for video rWpuh6dTiyY...
2024-10-16 15:11:52,845 - INFO - Finished scraping comments for video rWpuh6dTiyY. Total comments: 6
2024-10-16 15:11:52,849 - INFO - Comments for video rWpuh6dTiyY saved to: youtube_data/youtube_comments_rWpuh6dTiyY.csv
2024-10-16 15:11:52,850 - INFO - Starting to scrape comments for video: EhjK7P3uboE
2024-10-16 15:11:52,975 - WARNING - Encountered 403 Forbidden with reason "commentsDisabled"
2024-10-16 15:11:52,976 - WARNING - Encount

2024-10-16 15:12:00,836 - INFO - Scraped 112 comments so far for video HrUjS0XArAU...
2024-10-16 15:12:01,160 - INFO - Scraped 262 comments so far for video HrUjS0XArAU...
2024-10-16 15:12:01,535 - INFO - Scraped 470 comments so far for video HrUjS0XArAU...
2024-10-16 15:12:01,771 - INFO - Scraped 522 comments so far for video HrUjS0XArAU...
2024-10-16 15:12:01,772 - INFO - Finished scraping comments for video HrUjS0XArAU. Total comments: 522
2024-10-16 15:12:01,784 - INFO - Comments for video HrUjS0XArAU saved to: youtube_data/youtube_comments_HrUjS0XArAU.csv
2024-10-16 15:12:01,785 - INFO - Starting to scrape comments for video: yTVAyBICoYA
2024-10-16 15:12:02,132 - INFO - Scraped 149 comments so far for video yTVAyBICoYA...
2024-10-16 15:12:02,503 - INFO - Scraped 299 comments so far for video yTVAyBICoYA...
2024-10-16 15:12:02,825 - INFO - Scraped 441 comments so far for video yTVAyBICoYA...
2024-10-16 15:12:03,153 - INFO - Scraped 505 comments so far for video yTVAyBICoYA...
2024-

...

2024-10-16 15:15:06,815 - INFO - Starting to scrape comments for video: HFy94OsMo8c
2024-10-16 15:15:07,376 - INFO - Scraped 119 comments so far for video HFy94OsMo8c...
2024-10-16 15:15:07,732 - INFO - Scraped 232 comments so far for video HFy94OsMo8c...
2024-10-16 15:15:08,420 - INFO - Scraped 362 comments so far for video HFy94OsMo8c...
2024-10-16 15:15:08,779 - INFO - Scraped 544 comments so far for video HFy94OsMo8c...
2024-10-16 15:15:08,780 - INFO - Finished scraping comments for video HFy94OsMo8c. Total comments: 544
2024-10-16 15:15:08,792 - INFO - Comments for video HFy94OsMo8c saved to: youtube_data/youtube_comments_HFy94OsMo8c.csv
2024-10-16 15:15:08,793 - INFO - Starting to scrape comments for video: GurEPrfmXu4
2024-10-16 15:15:09,190 - INFO - Scraped 113 comments so far for video GurEPrfmXu4...
2024-10-16 15:15:09,973 - INFO - Scraped 218 comments so far for video GurEPrfmXu4...
2024-10-16 15:15:10,514 - INFO - Scraped 355 comments so far for video GurEPrfmXu4...
2024-10

2024-10-16 15:15:19,536 - INFO - Scraped 0 comments so far for video uJq_Z0dKI6w...
2024-10-16 15:15:19,537 - INFO - Finished scraping comments for video uJq_Z0dKI6w. Total comments: 0
2024-10-16 15:15:19,542 - INFO - Comments for video uJq_Z0dKI6w saved to: youtube_data/youtube_comments_uJq_Z0dKI6w.csv
2024-10-16 15:15:19,544 - INFO - Starting to scrape comments for video: JjsaQYHQqbA
2024-10-16 15:15:20,030 - INFO - Scraped 100 comments so far for video JjsaQYHQqbA...
2024-10-16 15:15:20,383 - INFO - Scraped 203 comments so far for video JjsaQYHQqbA...
2024-10-16 15:15:20,871 - INFO - Scraped 313 comments so far for video JjsaQYHQqbA...
2024-10-16 15:15:21,302 - INFO - Scraped 436 comments so far for video JjsaQYHQqbA...
2024-10-16 15:15:21,822 - INFO - Scraped 551 comments so far for video JjsaQYHQqbA...
2024-10-16 15:15:21,823 - INFO - Finished scraping comments for video JjsaQYHQqbA. Total comments: 551
2024-10-16 15:15:21,836 - INFO - Comments for video JjsaQYHQqbA saved to: yout

2024-10-16 15:15:34,004 - INFO - Scraped 313 comments so far for video 4Ck6XHPANJA...
2024-10-16 15:15:34,488 - INFO - Scraped 416 comments so far for video 4Ck6XHPANJA...
2024-10-16 15:15:34,813 - INFO - Scraped 502 comments so far for video 4Ck6XHPANJA...
2024-10-16 15:15:34,814 - INFO - Finished scraping comments for video 4Ck6XHPANJA. Total comments: 502
2024-10-16 15:15:34,825 - INFO - Comments for video 4Ck6XHPANJA saved to: youtube_data/youtube_comments_4Ck6XHPANJA.csv
2024-10-16 15:15:34,826 - INFO - Starting to scrape comments for video: xak6UQUpAhA
2024-10-16 15:15:35,120 - INFO - Scraped 1 comments so far for video xak6UQUpAhA...
2024-10-16 15:15:35,121 - INFO - Finished scraping comments for video xak6UQUpAhA. Total comments: 1
2024-10-16 15:15:35,126 - INFO - Comments for video xak6UQUpAhA saved to: youtube_data/youtube_comments_xak6UQUpAhA.csv
2024-10-16 15:15:35,127 - INFO - Starting to scrape comments for video: fc5_m2bqa-I
2024-10-16 15:15:35,464 - INFO - Scraped 42 co

2024-10-16 15:15:42,565 - INFO - Finished scraping comments for video K5LVekTMU40. Total comments: 4
2024-10-16 15:15:42,570 - INFO - Comments for video K5LVekTMU40 saved to: youtube_data/youtube_comments_K5LVekTMU40.csv
2024-10-16 15:15:42,571 - INFO - Starting to scrape comments for video: 7HLqtRakfgA
2024-10-16 15:15:42,712 - INFO - Scraped 0 comments so far for video 7HLqtRakfgA...
2024-10-16 15:15:42,714 - INFO - Finished scraping comments for video 7HLqtRakfgA. Total comments: 0
2024-10-16 15:15:42,723 - INFO - Comments for video 7HLqtRakfgA saved to: youtube_data/youtube_comments_7HLqtRakfgA.csv
2024-10-16 15:15:42,724 - INFO - Starting to scrape comments for video: 1JbcfnuOLeA
2024-10-16 15:15:42,859 - INFO - Scraped 0 comments so far for video 1JbcfnuOLeA...
2024-10-16 15:15:42,860 - INFO - Finished scraping comments for video 1JbcfnuOLeA. Total comments: 0
2024-10-16 15:15:42,864 - INFO - Comments for video 1JbcfnuOLeA saved to: youtube_data/youtube_comments_1JbcfnuOLeA.csv
2

2024-10-16 15:15:49,223 - INFO - Scraped 71 comments so far for video OrVrFhE6lAM...
2024-10-16 15:15:49,224 - INFO - Finished scraping comments for video OrVrFhE6lAM. Total comments: 71
2024-10-16 15:15:49,229 - INFO - Comments for video OrVrFhE6lAM saved to: youtube_data/youtube_comments_OrVrFhE6lAM.csv
2024-10-16 15:15:49,230 - INFO - Starting to scrape comments for video: kpI9DCrd-h4
2024-10-16 15:15:49,488 - INFO - Scraped 7 comments so far for video kpI9DCrd-h4...
2024-10-16 15:15:49,490 - INFO - Finished scraping comments for video kpI9DCrd-h4. Total comments: 7
2024-10-16 15:15:49,494 - INFO - Comments for video kpI9DCrd-h4 saved to: youtube_data/youtube_comments_kpI9DCrd-h4.csv
2024-10-16 15:15:49,496 - INFO - Starting to scrape comments for video: ksh0mao2L4Y
2024-10-16 15:15:49,666 - INFO - Scraped 4 comments so far for video ksh0mao2L4Y...
2024-10-16 15:15:49,667 - INFO - Finished scraping comments for video ksh0mao2L4Y. Total comments: 4
2024-10-16 15:15:49,671 - INFO - Co

In [7]:
videos_df

,video_id,title,description,published_at,channel_title,like_count,view_count
0,PIErb-yNEPw,How Does Race Impact Voting In the US? | US El...,How Does Race Impact Voting in The US? | US El...,2024-10-15T11:45:06Z,Firstpost,99,6220
1,q4R7KNr8FsU,LIVE | Trump Latest News | Trump&#39;s Explosi...,LIVE | Trump Latest News | Trump's Explosive T...,2024-10-15T08:35:16Z,CNN-News18,341,25205
2,qK2Zn5oCCq8,LIVE | Kamala Harris Slams Donald Trump In Pen...,US Democratic presidential candidate Kamala Ha...,2024-10-15T08:40:06Z,CNN-News18,302,29326
3,z1XLk5Ba0qc,"US Election Final Stretch | October 14, 2024","Today on CNN 10, we check in on the Presidenti...",2024-10-13T23:00:36Z,CNN 10,1114,73454
4,QHjptGT8mGc,Trump Live | Trump Attacks Iran | Trump Speech...,Donald Trump has been briefed by US intelligen...,2024-10-14T13:27:26Z,CNN-News18,12152,1352288
...,...,...,...,...,...,...,...
490,N4pErHCWHrU,What do UK students think about the US electio...,,2024-10-13T14:22:21Z,The Objective Hour with Tom Wan,4,438
491,KdhNbqJIXko,us election campaign 2024 |Trum and harris cam...,us election campaign 2024 |Trum and harris cam...,2024-10-15T12:35:46Z,NSP News Network,13,751
492,v-MjLB0j6I4,US Presidential Elections 2024 | Donald Trump ...,US Presidential Elections 2024 | Donald Trump ...,2024-10-14T07:06:35Z,Wild Wolf Focus,11,909
493,cznL69iqOrI,Swing States Polls Trump vs Harris #trump #har...,Swing States Polls Trump vs Harris #trump #har...,2024-10-14T17:00:27Z,Orbital Order,7,2999


In [8]:
comments_df

,comment_id,author,text,published_at,like_count,is_reply,parent_id
0,UgwprpzOqCA5C1xA-w14AaABAg,@dennysbastos-9026,Mundo 🌍vei sem porteira 🌍🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🌍✍🏻...,2024-10-13T07:59:25Z,0,False,LBKdco7sr-E
1,UgzOGdeyfS-xIyzommZ4AaABAg,@chadgaming8288,"If, or rather, WHEN Donald Trump wins the Elec...",2024-10-12T17:47:42Z,0,False,LBKdco7sr-E
2,Ugy9LK10g1PZAcc6qxV4AaABAg,@KAZVorpal,You can&#39;t trust anyone who says &quot;Cand...,2024-10-12T15:47:45Z,1,False,LBKdco7sr-E
3,UgxMhQmktBFjrB9syn94AaABAg,@KJSvitko,Trump is a con man fleecing the gullible and m...,2024-10-12T11:50:42Z,0,False,LBKdco7sr-E
4,UgxvPuI-2BNioLbp1454AaABAg,@DrMoto182,It takes a special kind of ignorance to want a...,2024-10-12T07:06:35Z,0,False,LBKdco7sr-E
...,...,...,...,...,...,...,...
497,UgwgO1VMJpyVyWpx0SR4AaABAg,@Forestventures,Im praying for President Trump&#39;s Victory 🙏...,2024-10-09T18:41:03Z,0,False,LBKdco7sr-E
498,UgypTF74PVGM3WHGLMx4AaABAg,@keithleeuwen877,So her Miller beer did nothing with that late ...,2024-10-09T18:39:39Z,0,False,LBKdco7sr-E
499,UgypTF74PVGM3WHGLMx4AaABAg.A9Nze-TJmqwA9OCuOA9sdx,@kelligonzales7783,It just shows to prove Harris is an alcoholic....,2024-10-09T20:44:12Z,0,True,UgypTF74PVGM3WHGLMx4AaABAg
500,UgypTF74PVGM3WHGLMx4AaABAg.A9Nze-TJmqwA9OI1tTjCU4,@carlagold8589,"Just proves she loves to drink , just like Nan...",2024-10-09T21:29:03Z,0,True,UgypTF74PVGM3WHGLMx4AaABAg
